In [29]:
import ee
import numpy as np
import matplotlib.pyplot as plt
import eedownload
import os
import gdal

ee.Initialize()

ImportError: libkea.so.1.4.7: cannot open shared object file: No such file or directory

In [3]:
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')

# Define date range
start_date = ee.Date('2018-01-01')
end_date = start_date.advance(180, 'day')    # 180 days later
sentinel1 = sentinel1.filterDate(start_date, end_date)

# Grab the VH and VV polarized parts in interferometric
# wide swath mode
vh = sentinel1.filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(
    ee.Filter.eq('instrumentMode', 'IW'))

# Separate data collected when the sat was ascending and descending
ascending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));
descending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));

# Create a composite from means at different polarizations
# and look angles *for visualization*
composite = ee.Image.cat([
    ascending.select('VH').max(),
    descending.select('VH').max(), 
    ee.ImageCollection(ascending.select('VV').merge(
        descending.select('VV'))).max()]).focal_median();

# Construct box to generate data for
centroid = [51.336665, 25.111551]
WIDTH = 0.05
HEIGHT = 0.05
llx = centroid[0] - WIDTH / 2
lly = centroid[1] - HEIGHT / 2
urx = centroid[0] + WIDTH / 2
ury = centroid[1] + HEIGHT / 2
area = ee.Geometry.Polygon([[llx,lly], [llx,ury], [urx,ury], [urx,lly]])

# path = image.getDownloadUrl({
# 		#'scale': 30,
# 		#'crs': 'EPSG:3310',
# 		#'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
# })

In [30]:
path = composite.getDownloadUrl({
#     'fileFormat': 'TFRecord',
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': str([[llx,lly], [llx,ury], [urx,ury], [urx,lly]])
});

In [31]:
path

'https://earthengine.googleapis.com/api/download?docid=05729f0941f060ba02275f8716fff932&token=4ef14edb4d7a608074b1691eee312fcf'

In [26]:
file = 'vh.tif'
ds = gdal.Open(file)
band = ds.GetRasterBand(1)
arr = band.ReadAsArray()